<a href="https://colab.research.google.com/github/olutobideyoung/MY-R-CODES/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt



In [ ]:
# Load your data
df = pd.read_csv("agriculture_climate_data.csv")  # replace with your actual file

# Compute 5-year moving averages for selected features
df['Humidity_MA5'] = df['Relative Humidity (%)'].rolling(window=5).mean()
df['Precip_MA5'] = df['Precipitation (mm)'].rolling(window=5).mean()
df['Land_MA5'] = df['Agricultural Land (sq. km)'].rolling(window=5).mean()

# Drop rows with NaNs resulting from moving average
df_clean = df.dropna().reset_index(drop=True)

In [ ]:
# Define features and target
X = df_clean[['Humidity_MA5', 'Precip_MA5', 'Land_MA5']]
y = df_clean['Gross Domestic Product (GDP) (Current LCU)']

# Optional: log-transform GDP if skewed
# y = np.log1p(y)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Train-test split (e.g., last 10 years as test)
split_index = int(len(X_scaled) * 0.85)
X_train, X_test = X_scaled[:split_index], X_scaled[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [ ]:
# Build the FNN model
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)  # Regression output
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=200, batch_size=8, validation_split=0.1, verbose=0)

In [ ]:
# Plot training history
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Loss Curve")
plt.show()

In [ ]:
# Evaluate
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae:.2f}")

# Predict and plot
y_pred = model.predict(X_test)

plt.figure(figsize=(10,5))
plt.plot(y_test.values, label='Actual GDP')
plt.plot(y_pred, label='Predicted GDP')
plt.title("Actual vs Predicted GDP (5-Year Moving Avg Inputs)")
plt.xlabel("Year Index")
plt.ylabel("GDP (Current LCU)")
plt.legend()
plt.show()